# W_GL_SEGMENT_DS ETL Process
### Oracle EBS FND_FLEX_VALUES to EDW W_GL_SEGMENT_DS
**ODI Package:** SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION  
**LKM:** BIAPPS SQL to Oracle (Multi Transport)  
**IKM:** BIAPPS Oracle Control Append

In [ ]:
%sql
-- Step 1: Variable IS_INCREMENTAL - Check if incremental load is enabled
CREATE OR REPLACE TEMP VIEW VAR_IS_INCREMENTAL AS
SELECT 
  CASE 
    WHEN COUNT(*) > 0 THEN 'Y'
    ELSE 'N'
  END AS IS_INCREMENTAL
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION'
  AND (DATASOURCE_NUM_ID = 380 OR DATASOURCE_NUM_ID = 380)
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'
  AND COMMITTED = '1'

In [ ]:
%sql
-- Step 2: Variable LAST_EXTRACT_DATE - Get last extraction date
CREATE OR REPLACE TEMP VIEW VAR_LAST_EXTRACT_DATE AS
SELECT
  COALESCE(
    date_format(MIN(LAST_MAX_DATE), 'yyyy-MM-dd HH:mm:ss'),
    '1970-01-01 00:00:00'
  ) AS LAST_EXTRACT_DATE
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION'
  AND DATASOURCE_NUM_ID = 380
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 3: Drop and Create Work Table C$_2624634_2_0 (as temp view)
DROP VIEW IF EXISTS C$_2624634_2_0;

CREATE OR REPLACE TEMP VIEW C$_2624634_2_0 AS
SELECT
  SQ_FND_FLEX_VALUES.CREATION_DATE,
  SQ_FND_FLEX_VALUES.LAST_UPDATE_DATE1,
  SQ_FND_FLEX_VALUES.CHANGED_ON_DT,
  SQ_FND_FLEX_VALUES.DELETE_FLG,
  CAST(SQ_FND_FLEX_VALUES.FLEX_VALUE_SET_ID AS STRING) AS SEGMENT_LOV_ID,
  COALESCE(SQ_FND_FLEX_VALUES.FLEX_VALUE, '__UNASSIGNED__') AS SEGMENT_VAL_CODE,
  CAST(SQ_FND_FLEX_VALUES.CREATED_BY AS STRING) AS CREATED_BY_ID,
  CAST(SQ_FND_FLEX_VALUES.LAST_UPDATED_BY AS STRING) AS CHANGED_BY_ID,
  to_date('1899-01-01', 'yyyy-MM-dd') AS SRC_EFF_FROM_DT,
  to_date('3714-01-01', 'yyyy-MM-dd') AS SRC_EFF_TO_DT,
  CAST(SQ_FND_FLEX_VALUES.FLEX_VALUE_SET_ID AS STRING) || '~' || SQ_FND_FLEX_VALUES.FLEX_VALUE AS INTEGRATION_ID
FROM (
  SELECT
    FND_FLEX_VALUE_SETS.FLEX_VALUE_SET_ID,
    CASE 
      WHEN FND_FLEX_VALUE_SETS.VALIDATION_TYPE = 'I' THEN FND_FLEX_VALUES.FLEX_VALUE
      ELSE FND_FLEX_VALUES.FLEX_VALUE || '~' || FND_FLEX_VALUES.PARENT_FLEX_VALUE_LOW
    END AS FLEX_VALUE,
    FND_FLEX_VALUES.LAST_UPDATED_BY,
    FND_FLEX_VALUES.CREATION_DATE,
    FND_FLEX_VALUES.CREATED_BY,
    FND_FLEX_VALUE_SETS.LAST_UPDATE_DATE AS LAST_UPDATE_DATE1,
    FND_FLEX_VALUES.LAST_UPDATE_DATE AS CHANGED_ON_DT,
    'N' AS DELETE_FLG
  FROM workspace.oracle_ebs.FND_FLEX_VALUES FND_FLEX_VALUES
  INNER JOIN workspace.oracle_ebs.FND_FLEX_VALUE_SETS FND_FLEX_VALUE_SETS
    ON FND_FLEX_VALUES.FLEX_VALUE_SET_ID = FND_FLEX_VALUE_SETS.FLEX_VALUE_SET_ID
  WHERE FND_FLEX_VALUE_SETS.VALIDATION_TYPE IN ('I', 'D')
    AND (
      FND_FLEX_VALUES.LAST_UPDATE_DATE > to_timestamp((SELECT LAST_EXTRACT_DATE FROM VAR_LAST_EXTRACT_DATE), 'yyyy-MM-dd HH:mm:ss')
      OR FND_FLEX_VALUE_SETS.LAST_UPDATE_DATE > to_timestamp((SELECT LAST_EXTRACT_DATE FROM VAR_LAST_EXTRACT_DATE), 'yyyy-MM-dd HH:mm:ss')
    )
) SQ_FND_FLEX_VALUES

In [ ]:
%sql
-- Step 4: Insert new rows into W_GL_SEGMENT_DS from work table
INSERT INTO workspace.oracle_edw.W_GL_SEGMENT_DS
(
  SEGMENT_LOV_ID,
  SEGMENT_VAL_CODE,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CREATED_ON_DT,
  CHANGED_ON_DT,
  AUX1_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  TENANT_ID,
  X_CUSTOM
)
SELECT
  SEGMENT_LOV_ID,
  SEGMENT_VAL_CODE,
  CREATED_BY_ID,
  CHANGED_BY_ID,
  CAST(CREATION_DATE AS TIMESTAMP) AS CREATED_ON_DT,
  CAST(CHANGED_ON_DT AS TIMESTAMP) AS CHANGED_ON_DT,
  CAST(LAST_UPDATE_DATE1 AS TIMESTAMP) AS AUX1_CHANGED_ON_DT,
  SRC_EFF_FROM_DT,
  SRC_EFF_TO_DT,
  DELETE_FLG,
  380 AS DATASOURCE_NUM_ID,
  INTEGRATION_ID,
  '1' AS TENANT_ID,
  '0' AS X_CUSTOM
FROM C$_2624634_2_0

In [ ]:
%sql
-- Step 5: Update reference dates in W_ETL_LOAD_DATES
MERGE INTO workspace.oracle_edw.W_ETL_LOAD_DATES AS TGT
USING (
  SELECT
    380 AS DATASOURCE_NUM_ID,
    'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION' AS PACKAGE_NAME,
    'W_GL_SEGMENT_DS' AS TARGET_TABLE_NAME,
    '__NOT_APPLICABLE__' AS ETL_USAGE_CODE,
    2624634 AS ETL_PROC_WID,
    12345 AS LOAD_PLAN_ID,
    date_add(current_timestamp(), -30) AS WIP_LOAD_START_DATE,
    current_timestamp() AS ETL_LOAD_DATE,
    (SELECT CASE WHEN IS_INCREMENTAL = 'Y' THEN '1' ELSE '0' END FROM VAR_IS_INCREMENTAL) AS COMMITTED
) SRC
ON TGT.DATASOURCE_NUM_ID = SRC.DATASOURCE_NUM_ID
  AND TGT.PACKAGE_NAME = SRC.PACKAGE_NAME
  AND TGT.ETL_USAGE_CODE = SRC.ETL_USAGE_CODE
WHEN MATCHED THEN
  UPDATE SET
    TGT.TARGET_TABLE_NAME = SRC.TARGET_TABLE_NAME,
    TGT.ETL_PROC_WID = SRC.ETL_PROC_WID,
    TGT.LOAD_PLAN_ID = SRC.LOAD_PLAN_ID,
    TGT.WIP_LOAD_START_DATE = SRC.WIP_LOAD_START_DATE,
    TGT.ETL_LOAD_DATE = SRC.ETL_LOAD_DATE,
    TGT.COMMITTED = SRC.COMMITTED
WHEN NOT MATCHED THEN
  INSERT (
    DATASOURCE_NUM_ID,
    PACKAGE_NAME,
    TARGET_TABLE_NAME,
    ETL_USAGE_CODE,
    ETL_PROC_WID,
    LOAD_PLAN_ID,
    WIP_LOAD_START_DATE,
    ETL_LOAD_DATE,
    COMMITTED
  )
  VALUES (
    SRC.DATASOURCE_NUM_ID,
    SRC.PACKAGE_NAME,
    SRC.TARGET_TABLE_NAME,
    SRC.ETL_USAGE_CODE,
    SRC.ETL_PROC_WID,
    SRC.LOAD_PLAN_ID,
    SRC.WIP_LOAD_START_DATE,
    SRC.ETL_LOAD_DATE,
    SRC.COMMITTED
  )

In [ ]:
%sql
-- Step 6: Insert reference dates to History Table W_ETL_LOAD_DATES_LOG
INSERT INTO workspace.oracle_edw.W_ETL_LOAD_DATES_LOG
(
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
)
SELECT
  DATASOURCE_NUM_ID,
  PACKAGE_NAME,
  TARGET_TABLE_NAME,
  ETL_USAGE_CODE,
  ETL_PROC_WID,
  LOAD_PLAN_ID,
  2624634 AS SESSION_ID,
  WIP_LOAD_START_DATE,
  LAST_MAX_DATE,
  ETL_LOAD_DATE,
  COMMITTED
FROM workspace.oracle_edw.W_ETL_LOAD_DATES
WHERE DATASOURCE_NUM_ID = 380
  AND PACKAGE_NAME = 'SDE_ORAR122_ADAPTOR_SDE_ORA_GLSEGMENTDIMENSION'
  AND ETL_USAGE_CODE = '__NOT_APPLICABLE__'

In [ ]:
%sql
-- Step 7: Drop work table (temp view)
DROP VIEW IF EXISTS C$_2624634_2_0;
DROP VIEW IF EXISTS VAR_IS_INCREMENTAL;
DROP VIEW IF EXISTS VAR_LAST_EXTRACT_DATE